# Librery

In [1]:
# Import the libraries
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense,  Conv1D, MaxPooling1D, Flatten

2023-12-20 07:49:24.445167: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 07:49:24.464317: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 07:49:24.464340: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 07:49:24.464357: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 07:49:24.468867: I tensorflow/core/platform/cpu_feature_g

# Make Dictionary for One-hot Encoding

In [7]:
# Import the training set
train_log = pd.read_csv('/user_data/my/作業/data/train_logs.csv')  # 讀取訓練集
train_log = train_log[["id","down_event"]]
train_log

,id,down_event
0,001519c8,Leftclick
1,001519c8,Leftclick
2,001519c8,Shift
3,001519c8,q
4,001519c8,q
...,...,...
8405893,fff05981,Leftclick
8405894,fff05981,Shift
8405895,fff05981,q
8405896,fff05981,Leftclick


In [8]:
train_score = pd.read_csv('/user_data/my/作業/data/train_scores.csv')  # 讀取訓練集label
train_score = train_score.set_index("id")
train_score

,score
id,
001519c8,3.5
0022f953,3.5
0042269b,6.0
0059420b,2.0
0075873a,4.0
...,...
ffb8c745,3.5
ffbef7e5,4.0
ffccd6fd,1.5


In [40]:
def oneHotDict(df_word_list):
    print(f"enter oneHotDict")
    
    unique_list = df_word_list.unique()
    list_len = len(unique_list)
    category_list = to_categorical(range(list_len))
    
    index_dict = {}
    for i in range(list_len):
        index_dict[unique_list[i]] = category_list[i]
        
    index_dict[0] = np.zeros(list_len)
    
    print(f"exit oneHotDict")
    return index_dict

# One-hot Encoding Data

In [16]:
def eventPerEssay(df):
    id_class = df.groupby('id')
    event_per_essay = [ [id, len(id_class.groups[id])] for id in train_log['id'].unique() ]
    event_per_essay = np.array(event_per_essay)
    return id_class, event_per_essay

In [51]:
def oneHot_eachEvent( df:pd.DataFrame, num_desire:int, index_dict:dict )->list:
    # 確保 r_df 會有 num_desire 筆 "down_event"
    r_df = df[["down_event"]]
    if(len(df)<num_desire):
        # print("enter padding")
        padding = { "down_event":["0"]*(num_desire-len(df)) }
        padding = pd.DataFrame(padding)
        r_df = pd.concat([r_df, padding], axis=0)

    r_df = r_df.reset_index().drop("index", axis=1)

    # 製作 r_log (return value)
    r_log = []
    for i in range(num_desire):
        event = r_df.at[i, "down_event"]
        r_log.append(index_dict[event])
        
    return r_log

In [54]:
def Doc2Inputs(doc_list:pd.DataFrame, score_list:pd.DataFrame=None)->(np.ndarray, np.ndarray):
    id_class, event_per_essay = eventPerEssay(doc_list)
    index_dict = oneHotDict(doc_list['down_event'])
    
    augmented_doc_list = []
    augmented_score_list = []
    for ind_essay, num_event in event_per_essay:
        print(f"ind_essay = {ind_essay}")
        cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
        if score_list!= None:
            cur_score = score_list.at[ind_essay,"score"]

        passage_size = 1000
        for ind_passage in range(int(num_event)//passage_size + 1):
            sub_cur_df = cur_df[ind_passage*1000:(ind_passage+1)*1000]
            augmented_doc_list.append( oneHot_eachEvent(sub_cur_df, passage_size, index_dict) )
            augmented_score_list.append([cur_score])
    
    augmented_doc_list   = np.array(augmented_doc_list)
    augmented_score_list = np.array(augmented_score_list).reshape(-1)
    
    return augmented_doc_list, augmented_score_list

In [55]:
arr_trainLog_2D_augment, arr_trainScore_2D_augment = Doc2Inputs(train_log, train_score)
np.save("/user_data/my/作業/data/trainScore_2D_augment.npy", arr_trainScore_2D_augment)
np.save("/user_data/my/作業/data/trainLog_2D_augment.npy", arr_trainLog_2D_augment)

enter oneHotDict
exit oneHotDict
ind_essay = 001519c8
ind_essay = 0022f953
ind_essay = 0042269b
ind_essay = 0059420b
ind_essay = 0075873a
ind_essay = 0081af50
ind_essay = 0093f095
ind_essay = 009e23ab
ind_essay = 00e048f1
ind_essay = 00e1f05a
ind_essay = 00e713bd
ind_essay = 00f0737e
ind_essay = 00f8e84c
ind_essay = 00fc9a6a
ind_essay = 0144e4d5
ind_essay = 014e7ae9
ind_essay = 015aa732
ind_essay = 0178a105
ind_essay = 0182aa1c
ind_essay = 0190ff4c
ind_essay = 01963e20
ind_essay = 019737b6
ind_essay = 01992d32
ind_essay = 01c359fc
ind_essay = 01d0ba4b
ind_essay = 01d602a7
ind_essay = 0249a095
ind_essay = 0262bf61
ind_essay = 026be946
ind_essay = 0294b4f5
ind_essay = 02a41d1a
ind_essay = 02cf6a52
ind_essay = 02d3c9fc
ind_essay = 02e86b6a
ind_essay = 031c0c58
ind_essay = 034d61db
ind_essay = 035f09fc
ind_essay = 036cfd42
ind_essay = 0395b217
ind_essay = 03971ddf
ind_essay = 040c429b
ind_essay = 0417d421
ind_essay = 0432f117
ind_essay = 0445b534
ind_essay = 044b274d
ind_essay = 044c5c54
i

# Building Model

In [2]:
trainScore_2D_augment = np.load("/user_data/my/作業/data/trainScore_2D_augment.npy")
trainLog_2D_augment   = np.load("/user_data/my/作業/data/trainLog_2D_augment.npy")

In [4]:
model1D = Sequential(
    [
        Conv1D(10, 3, padding="same", activation='linear', name = "conv_find"),
        MaxPooling1D(3, padding="same", strides=1, name = "pool_find"),
        Conv1D( 5, 3, padding="same", activation='linear', name = "conv_pattern"),
        MaxPooling1D(10, name="pool_pattern"),
        Conv1D( 5, 3, padding="same", activation='linear', name = "conv_contigious"),
        MaxPooling1D(10, name="pool_contigious"),
        Flatten(name="flatten"),
        Dense(10, activation='linear', name="dense_nn"),
        BatchNormalization(name = "BatchNormalization"),
        Dense(1, activation='linear', name="dense_scoring")
    ],
    "shortCut_vs_score_1D"
)
input_shape = (None, 1000, 131)
model1D.build(input_shape)

2023-12-20 07:49:30.826942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 07:49:30.841019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 07:49:30.841117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
model1D.summary()

Model: "shortCut_vs_score_1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_find (Conv1D)          (None, 1000, 10)          3940      
                                                                 
 pool_find (MaxPooling1D)    (None, 1000, 10)          0         
                                                                 
 conv_pattern (Conv1D)       (None, 1000, 5)           155       
                                                                 
 pool_pattern (MaxPooling1D  (None, 100, 5)            0         
 )                                                               
                                                                 
 conv_contigious (Conv1D)    (None, 100, 5)            80        
                                                                 
 pool_contigious (MaxPoolin  (None, 10, 5)             0         
 g1D)                                         

In [6]:
model1D.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mean_squared_error']
              )

In [8]:
batch_size = 10
epochs = 10
verbose = 1

In [9]:
model1D.fit(
    x=trainLog_2D_augment,
    y=trainScore_2D_augment,
    batch_size=batch_size,
    epochs=epochs,
    verbose=verbose,
    validation_split=0.1,
    shuffle=True
)

Epoch 1/10


2023-12-20 08:08:11.279991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8900
2023-12-20 08:08:12.755396: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-20 08:08:15.095637: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-20 08:08:15.999854: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb114c69660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-20 08:08:15.999936: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-12-20 08:08:16.093275: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-20 08:08:16.494559: I ./tensorflow/compiler/jit/device_comp

868/868 [==============================] - 17s 9ms/step - loss: 4.8506 - mean_squared_error: 4.8506 - val_loss: 1.2014 - val_mean_squared_error: 1.2014
Epoch 2/10
868/868 [==============================] - 7s 8ms/step - loss: 0.9085 - mean_squared_error: 0.9085 - val_loss: 1.0083 - val_mean_squared_error: 1.0083
Epoch 3/10
868/868 [==============================] - 7s 8ms/step - loss: 0.8230 - mean_squared_error: 0.8230 - val_loss: 0.9973 - val_mean_squared_error: 0.9973
Epoch 4/10
868/868 [==============================] - 6s 7ms/step - loss: 0.7971 - mean_squared_error: 0.7971 - val_loss: 0.8774 - val_mean_squared_error: 0.8774
Epoch 5/10
868/868 [==============================] - 7s 8ms/step - loss: 0.7674 - mean_squared_error: 0.7674 - val_loss: 0.9998 - val_mean_squared_error: 0.9998
Epoch 6/10
868/868 [==============================] - 7s 8ms/step - loss: 0.7636 - mean_squared_error: 0.7636 - val_loss: 2.4736 - val_mean_squared_error: 2.4736
Epoch 7/10
868/868 [==================